In [10]:
import cv2
import numpy as np
import onnxruntime as ort
import urllib.request

# URL do modelo pré-treinado
model_url = "https://github.com/Tencent/ncnn/releases/download/v1.0.0/openface.onnx"

# Caminho para o arquivo do modelo pré-treinado
model_path = "caminho/para/openface.onnx" # substitua pelo caminho correto

# Faz o download do modelo pré-treinado e o salva em um arquivo temporário
#with tempfile.NamedTemporaryFile(suffix=".onnx") as tmp_file:
#    urllib.request.urlretrieve(model_url, tmp_file.name)
#    model_path = tmp_file.name

# Carrega o modelo pré-treinado de reconhecimento facial
session = ort.InferenceSession(model_path)

# Captura imagens da câmera
video = cv2.VideoCapture(0)

# Define o tamanho da janela
window_name = "Face recognition"
cv2.namedWindow(window_name)

# Define a região de interesse (ROI) para detecção facial
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Loop para processar cada frame do vídeo
while True:
    # Captura um frame do vídeo
    ret, frame = video.read()
    
    # Converte a imagem para escala de cinza
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detecta faces na imagem
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Loop para processar cada face detectada na imagem
    for (x, y, w, h) in faces:
        # Define a região de interesse (ROI) para a face detectada
        roi = gray[y:y+h, x:x+w]
        
        # Redimensiona a ROI para o tamanho esperado pelo modelo
        roi_resized = cv2.resize(roi, (96, 96))
        
        # Normaliza os valores dos pixels da imagem
        roi_norm = roi_resized / 255.0
        
        # Transforma a imagem em um tensor
        roi_tensor = np.expand_dims(np.expand_dims(roi_norm, axis=0), axis=-1).astype(np.float32)
        
        # Realiza a inferência do modelo na imagem
        outputs = session.run(None, {'input_1': roi_tensor})

        # Mostra o resultado da inferência na imagem
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, "Face", (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        cv2.putText(frame, "OpenFace Embedding: " + str(outputs[0]), (x, y+h+20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Mostra a imagem na janela
    cv2.imshow(window_name, frame)

    # Espera por uma tecla
    key = cv2.waitKey(1)

    # Sai do loop se a tecla 'Esc' for pressionada
    if key == 27:
        break

# Libera a câmera e fecha todas as j


NoSuchFile: [ONNXRuntimeError] : 3 : NO_SUCHFILE : Load model from caminho/para/openface.onnx failed:Load model caminho/para/openface.onnx failed. File doesn't exist